### SPLIT AFRICA DATASET 128

I re-ran this notebook until the split was reasonably even between train/validation/test. In a perfect split the `low-med-high rfracs` would all equal 1 and the `block split fracs` would equal the respective `group fracs`.  I am not too concerned with less validation data, but prefer that `water-tag=low` be well represented in both test and validation.

---

In [1]:
import pandas as pd
from random import sample

In [2]:
WSIZE=128
DSET=f'surface-water.africa.win{WSIZE}'
NO_DATA_MAX=0.05
IDENTS=['group_id','window_index']

In [3]:
df=pd.read_csv(f'{DSET}.csv')
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=IDENTS).shape[0])
df=df[df.no_data<=NO_DATA_MAX]
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=IDENTS).shape[0])
df.sample(3)

24762 684 24338
21278 644 20929


,gsw_path,s1_path,block_id,group_id,window_index,window,water,not_water,no_data,dataset
21563,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_18_-33,group_18.17_-33.29,3,"(0, 384, 128, 128)",0.513733,0.444702,0.041565,africa_mtn
21143,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_20_-34,group_20.42_-34.06,7,"(128, 384, 128, 128)",0.006531,0.993469,0.000000,africa_mtn
5063,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_25_-27,group_25.08_-27.97,14,"(384, 256, 128, 128)",0.140076,0.849670,0.010254,africa


---

In [4]:
def water_counter(df,quantiles=[0.15,0.85],labels=['low','medium','high']):
    wqs=[0]+[df.water.quantile(q) for q in quantiles]+[1+1e-8]
    dfs=[]
    for i,l in enumerate(labels):
        _df=df.copy()[(df.water>=wqs[i])&(df.water<wqs[i+1])]
        _df['water_tag']=l
        dfs.append(_df)
    return pd.concat(dfs).sample(frac=1)


def split_groups(blocks,valid_frac=0.2,test_frac=0.1,hold_out_frac=0.2,relative_frac=True):
    if relative_frac:
        valid_frac=(1-hold_out_frac)*valid_frac
        test_frac=(1-hold_out_frac)*test_frac
    nb_blocks=len(blocks)
    nb_valid=int(valid_frac*nb_blocks)    
    nb_test=int(test_frac*nb_blocks)
    nb_hold_out=int(hold_out_frac*nb_blocks)
    blocks=sample(blocks,nb_blocks)
    valid=blocks[:nb_valid]
    test=blocks[nb_valid:nb_valid+nb_test]
    hold_out=blocks[nb_valid+nb_test:nb_valid+nb_test+nb_hold_out]
    train=blocks[nb_valid+nb_test+nb_hold_out:]
    rtotal=len(train)+len(valid)+len(test)
    print('block relative split:',len(train)/rtotal,len(valid)/rtotal,len(test)/rtotal,len(hold_out)/nb_blocks)
    print('block split fracs:',len(train)/nb_blocks,len(valid)/nb_blocks,len(test)/nb_blocks,len(hold_out)/nb_blocks)
    return train, valid, test, hold_out


def sub_df(typ,df,blocks,cnt):
    
    sdf=df.copy()[df.block_id.isin(blocks)]
    sdf['data_split']=typ
    
    sgcnt=sdf.drop_duplicates(subset=IDENTS).shape[0]
    frac=sgcnt/cnt
    
    ltcnt=df[df.water_tag=='low'].shape[0]*frac
    mtcnt=df[df.water_tag=='medium'].shape[0]*frac
    htcnt=df[df.water_tag=='high'].shape[0]*frac
    
    sltcnt=sdf[sdf.water_tag=='low'].shape[0]
    smtcnt=sdf[sdf.water_tag=='medium'].shape[0]
    shtcnt=sdf[sdf.water_tag=='high'].shape[0]
    print()
    print(f'{typ} group:',frac,f'({sgcnt}/{cnt})  [{int(ltcnt)},{int(mtcnt)},{int(htcnt)}]')
    print('* low-med-high rfracs:',sltcnt/ltcnt,smtcnt/mtcnt,shtcnt/htcnt)
    return sdf
    
    
def df_splitter(df,train, valid, test, hold_out):
    cnt=df.drop_duplicates(subset=IDENTS).shape[0]
    tdf=sub_df('train',df,train,cnt)    
    vdf=sub_df('valid',df,valid,cnt)    
    sdf=sub_df('test',df,test,cnt)    
    hdf=sub_df('hold_out',df,hold_out,cnt)
    df=pd.concat([tdf, vdf, sdf, hdf])
    return df

---

In [5]:
df=water_counter(df)

In [6]:
blocks=list(df.block_id.unique())

In [7]:
print('-'*85)
train, valid, test, hold_out=split_groups(blocks)
print('-'*85)
df=df_splitter(df, train, valid, test, hold_out)

-------------------------------------------------------------------------------------
block relative split: 0.7015503875968992 0.1996124031007752 0.09883720930232558 0.19875776397515527
block split fracs: 0.562111801242236 0.15993788819875776 0.07919254658385093 0.19875776397515527
-------------------------------------------------------------------------------------

train group: 0.5799608199149505 (12138/20929)  [1839,8650,1851]
* low-med-high rfracs: 1.0005134374227551 1.0198707599227501 0.9199264587155812

valid group: 0.1500310573844904 (3140/20929)  [475,2237,478]
* low-med-high rfracs: 1.0677911085132583 0.98806226792589 0.9918581286017593

test group: 0.07664006880405179 (1604/20929)  [243,1143,244]
* low-med-high rfracs: 0.9134837928829771 0.9876766765620311 1.1241232367077294

hold_out group: 0.19336805389650724 (4047/20929)  [613,2884,617]
* low-med-high rfracs: 0.9801521650720714 0.95454900063377 1.1972830364767861


---

In [8]:
df.sample(3)

,gsw_path,s1_path,block_id,group_id,window_index,window,water,not_water,no_data,dataset,water_tag,data_split
22502,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_31_-28,group_31.45_-28.76,6,"(128, 256, 128, 128)",0.355713,0.644287,0.000000,africa_mtn,high,train
14574,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_28_-29,group_28.77_-29.67,11,"(256, 384, 128, 128)",0.068237,0.931763,0.000000,africa,medium,train
4572,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_18_7,group_18.07_7.08,6,"(128, 256, 128, 128)",0.114197,0.841309,0.044495,africa,medium,train


In [9]:
file_name=f'{DSET}.split.csv'
df.to_csv(file_name,index=False)
print(f'UPLOAD: gsutil cp {file_name} gs://surface-water-public/data/v1/datasets')

UPLOAD: gsutil cp surface-water.africa.win128.split.csv gs://surface-water-public/data/v1/datasets


---

In [15]:
# !gsutil cp surface-water.africa.win128.split.csv gs://surface-water-public/data/v1/datasets

---

##### WATER/NOT RATIO

In [11]:
_df=df[df.data_split.isin(['train','valid'])]

In [12]:
_df.not_water.mean()/_df.water.mean()

7.067384919140698

In [13]:
_df.drop_duplicates(subset=IDENTS)[['water','not_water']].describe()

,water,not_water
count,15278.000000,15278.000000
mean,0.122895,0.872131
std,0.138083,0.138971
min,0.005005,0.370117
25%,0.024353,0.826477
50%,0.066711,0.927734
75%,0.168320,0.970520
max,0.599976,0.994995
